In [1]:
import pandas as pd
import numpy as np
import os
import ast
import statistics
import re

In [21]:

scriptDirectory = r"C:\\Users\\ptut0\\Documents\\speech_timing\\basline\\data\\txt\\clean\\"
writeDirectory = "C:\\Users\\ptut0\\Documents\\speech_timing\\basline\\data\\"
ssmlTags = ['x-weak', 'weak', 'medium-p', 'strong', 'x-strong']
pausePunct = [',', '!', '?', '.']
lenghtData = pd.read_csv('C:\\Users\\ptut0\\Documents\\speech_timing\\basline\\data\\clean_baseline.csv')

In [26]:
data = pd.read_csv('C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\train\\word_length-copy.csv')
audioDirectory = r"C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\train\\wav\\"
names =[]
for file in os.listdir(audioDirectory):
  filename = file.split('.')[0]
  if filename not in data['filename'].values:
    names.append(filename)
print(len(names))
  

699


In [11]:
import pandas as pd
import numpy as np
import os
import ast
import statistics

writeDirectory = "C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\train\\"
lenghtData = pd.read_csv('C:\\Users\\ptut0\\Documents\\speech_timing\TEDLIUM_release-3\\speaker-adaptation\\train\\chunk_length.csv')
ssmlTags = ['x-weak', 'weak', 'medium-p', 'strong', 'x-strong']
pausePunct = [',', '!', '?', '.']

for index, row in lenghtData.iterrows():
  dataframe = pd.DataFrame({'filename' : [], 'discrete_pauses' : []})
  binLengths = {'filename': row['filename'], 'discrete_pauses' : []}
  # read string lists to real lists
  pauseLengths = ast.literal_eval(row['pause_lengths'])
  script = row['script']
  # uncomment below if running from txt scripts
  #with open(script_location, "r") as f:
  #  script = f.read()
  scriptList = script.split()
  # Only keep pauses at break punctuation
  naCount = pauseLengths.count("NA")
  if naCount < (len(pauseLengths)/4):
    if len(scriptList)-1 == len(pauseLengths):
      # interpolate NA values to mean
      lenMean = statistics.fmean(list(filter(lambda length: length != "NA", pauseLengths)))
      for i,pause in enumerate(pauseLengths):
        if pause == "NA":
          pauseLengths[i] = lenMean
      # get only non zero pentiles
      nonzPauseLengths = [i for i in pauseLengths if i > 0]
      pentiles = np.quantile(nonzPauseLengths, [0,0.2,0.4,0.6,0.8,1])
      # bin by pentiles
      pause_punct = []
      for i, pauseLength in enumerate(pauseLengths):
        if any(e in scriptList[i] for e in pausePunct):
          pause_punct.append(pauseLength)
          if pauseLength <= pentiles[1]:
            binLengths['discrete_pauses'].append(ssmlTags[0])
          elif pauseLength <= pentiles[2]:
            binLengths['discrete_pauses'].append(ssmlTags[1])
          elif pauseLength <= pentiles[3]:
            binLengths['discrete_pauses'].append(ssmlTags[2])
          elif pauseLength <= pentiles[4]:
            binLengths['discrete_pauses'].append(ssmlTags[3])
          elif pauseLength > pentiles[4]:
            binLengths['discrete_pauses'].append(ssmlTags[4])
          else:
            binLengths['discrete_pauses'] = ["ERROR-BIN"]
            dataframe = pd.concat([dataframe, pd.Series(binLengths).to_frame().T], ignore_index=True)
            dataframe.to_csv(writeDirectory+"discrete_pause_length.csv", mode='a', header=not os.path.exists(writeDirectory+"discrete_pause_length.csv"))
            continue
    else:
      binLengths['discrete_pauses'] = ["ERROR-MISMATCH"]
  else:
    binLengths['discrete_pauses'] = ["ERROR-NA"]
  dataframe = pd.concat([dataframe, pd.Series(binLengths).to_frame().T], ignore_index=True)
  dataframe.to_csv(writeDirectory+"discrete_pause_length.csv", mode='a', header=not os.path.exists(writeDirectory+"discrete_pause_length.csv"))

In [13]:
import pandas as pd
import numpy as np
import os
import ast
import statistics

writeDirectory = "C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\dev\\"
ssmlTags = ['x-slow', 'slow', 'medium', 'fast', 'x-fast']
lengthData = pd.read_csv('C:\\Users\\ptut0\\Documents\\speech_timing\TEDLIUM_release-3\\speaker-adaptation\\dev\\chunk_length.csv')

def syllable_count(word):
  word = word.lower()
  count = 0
  vowels = "aeiouy"
  if word[0] in vowels:
    count += 1
  # a syllable is the number is distinct vowels
  for index in range(1, len(word)):
    if word[index] in vowels and word[index - 1] not in vowels:
      count += 1
  # silent e at the end of a word to not contribute to vowels
  if word.endswith("e"):
    count -= 1
  # silent e at the end of a word to not contribute to vowels
  if word.endswith("e's"):
    count -= 1
  # ending in ed does not contribute
  if word.endswith("ed"):
    count -= 1 
  # 'nt words have an extra syllable
  if "'nt" in word:
    count += 1
  if count == 0:
    count += 1
  return count

binLengths = []
for index, row in lengthData.iterrows():
  discrete_lengths = []
  # read string lists as real lists
  wordLengths = ast.literal_eval(row['word_lengths'])
  # if script is in the data file
  script = row['script']
  # uncomment below if running from txt scripts
  #with open(script_location, "r") as f:
  #  script = f.read()
  # need just the words as a list
  cleanScript = script.replace(",",'')
  cleanScript = cleanScript.replace(".",'')
  scriptList = cleanScript.split()
  # linear transformation relative to syllables
  # the lables are the pentiles (5 bins) specific to the speaker
  # interpolate NAs, if more than 1/4 of words are NA flag error
  naCount = wordLengths.count("NA")
  if naCount < (len(wordLengths)/4):
    # to simplify I changed hypens to spaces
    # to make robust to hypens need to adjust the pause and aligner code
    #if len(scriptList) != len(wordLengths):
    #  hyphens = [i for i, s in enumerate(scriptList) if "-" in s]
    #  if hyphens:
    #    for hyphen in hyphens:
    #      if wordLengths[hyphen+1] != "NA":
    #        if wordLengths[hyphen] != "NA":
    #          wordLengths[hyphen] = wordLengths[hyphen] + wordLengths[hyphen+1]
    #          del wordLengths[hyphen+1]
    #        else:
    #          del wordLengths[hyphen]
    #      else:
    #       del wordLengths[hyphen+1]
    if len(scriptList) != len(wordLengths):
      binLengths.append(["ERROR-MISMATCH"])
      continue
    # interpolate NA values to mean
    lenMean = statistics.fmean(list(filter(lambda length: length != "NA", wordLengths)))
    for i,word in enumerate(wordLengths):
      if word == "NA":
        wordLengths[i] = lenMean
    # make all word lengths relative to sylls
    syllRelLength = [length / syllable_count(scriptList[word]) for word,length in enumerate(wordLengths)]

    pentiles = np.quantile(syllRelLength, [0,0.2,0.4,0.6,0.8,1])
    # bin by pentiles
    for wordLength in syllRelLength:
      if wordLength <= pentiles[1]:
        discrete_lengths.append(ssmlTags[4])
      elif wordLength <= pentiles[2]:
        discrete_lengths.append(ssmlTags[3])
      elif wordLength <= pentiles[3]:
        discrete_lengths.append(ssmlTags[2])
      elif wordLength <= pentiles[4]:
        discrete_lengths.append(ssmlTags[1])
      elif wordLength > pentiles[4]:
        discrete_lengths.append(ssmlTags[0])
      else:
        discrete_lengths.append(["ERROR-BIN"])
        continue
  else:
    binLengths.append(["ERROR-NA"])
    continue
  binLengths.append(discrete_lengths)

lengthData["discrete_lengths"] = binLengths
#opening in excel is breaking the csv so append to it
lengthData.to_csv(writeDirectory+"discrete_word_length.csv", mode='a', header=not os.path.exists(writeDirectory+"discrete_word_length.csv"))


In [9]:
from collections import Counter
Counter(['medium', 'medium', 'medium', 'slow', 'medium', 'medium', 'fast', 'fast', 'slow', 'fast', 'medium', 'medium', 'fast', 'fast', 'slow', 'medium', 'medium', 'medium', 'medium', 'fast', 'medium', 'medium', 'slow', 'fast', 'fast', 'fast', 'medium', 'fast', 'fast', 'medium', 'fast', 'fast', 'fast', 'fast', 'medium', 'fast', 'medium', 'slow', 'fast', 'slow', 'slow', 'slow', 'slow', 'fast', 'slow', 'fast', 'medium', 'fast', 'fast', 'slow', 'slow', 'slow', 'fast', 'slow', 'fast', 'fast', 'fast', 'medium', 'fast', 'medium', 'fast', 'medium', 'slow', 'medium', 'fast', 'medium', 'slow', 'fast', 'fast', 'medium', 'fast', 'fast', 'medium', 'medium', 'medium', 'fast', 'slow', 'fast', 'medium', 'slow', 'fast', 'medium', 'slow', 'fast', 'fast', 'fast', 'medium', 'fast', 'fast', 'slow', 'medium', 'medium', 'slow', 'fast', 'slow', 'slow', 'fast', 'slow', 'medium', 'slow', 'fast', 'fast', 'fast', 'medium', 'x-slow', 'fast', 'medium', 'slow', 'fast', 'medium', 'fast', 'medium', 'medium', 'fast', 'slow', 'medium', 'medium', 'slow', 'medium', 'medium', 'fast', 'medium', 'medium', 'x-fast', 'x-fast', 'slow', 'slow', 'medium', 'medium', 'slow', 'slow', 'slow', 'fast', 'x-slow', 'slow', 'slow', 'medium', 'fast', 'slow', 'fast', 'slow', 'medium', 'medium', 'fast', 'slow', 'slow', 'medium', 'medium', 'medium', 'medium', 'fast', 'slow', 'medium', 'fast', 'medium', 'medium', 'slow', 'fast', 'fast', 'medium', 'slow', 'medium', 'fast', 'fast', 'medium', 'fast', 'medium', 'medium', 'medium', 'medium', 'medium', 'slow', 'fast', 'fast', 'slow', 'medium', 'fast', 'medium', 'medium', 'fast', 'slow', 'slow', 'slow', 'fast', 'fast', 'fast', 'medium', 'slow', 'fast', 'medium', 'medium', 'medium', 'slow', 'fast', 'slow', 'fast', 'slow', 'slow', 'medium', 'slow', 'medium', 'fast', 'medium', 'fast', 'medium', 'medium', 'fast', 'fast', 'slow', 'medium', 'slow', 'medium', 'medium', 'x-fast', 'fast', 'medium', 'slow', 'slow', 'medium', 'medium', 'slow', 'fast', 'fast', 'slow', 'fast', 'medium', 'slow', 'slow', 'fast', 'fast', 'slow', 'fast', 'fast', 'slow', 'slow', 'medium', 'fast', 'slow', 'fast', 'medium', 'fast', 'medium', 'fast', 'medium', 'medium', 'fast', 'medium', 'medium', 'slow', 'fast', 'fast', 'slow', 'medium', 'fast', 'slow', 'medium', 'slow', 'medium', 'fast', 'slow', 'slow', 'slow', 'fast', 'slow', 'slow', 'slow', 'fast', 'medium', 'fast', 'medium', 'slow', 'fast', 'slow', 'medium', 'slow', 'fast', 'slow', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'medium', 'fast', 'slow', 'x-slow', 'medium', 'slow', 'fast', 'slow', 'slow', 'medium', 'slow', 'slow', 'fast', 'medium', 'slow', 'medium', 'medium', 'medium', 'medium', 'slow', 'slow', 'medium', 'medium', 'slow', 'fast', 'fast', 'fast', 'medium', 'medium', 'slow', 'medium', 'fast', 'slow', 'medium', 'medium', 'medium', 'fast', 'medium', 'medium', 'slow', 'medium', 'medium', 'slow', 'medium', 'slow', 'medium', 'medium', 'slow', 'medium', 'fast', 'medium', 'slow', 'fast', 'slow', 'medium', 'medium', 'fast', 'slow', 'fast', 'fast', 'fast', 'medium', 'slow', 'slow', 'medium', 'x-fast', 'slow', 'fast', 'fast', 'medium', 'medium', 'fast', 'slow', 'fast', 'fast', 'slow', 'slow', 'slow', 'medium', 'medium', 'medium', 'slow', 'fast', 'fast', 'slow', 'fast', 'fast', 'fast'])

Counter({'medium': 136, 'slow': 107, 'fast': 121, 'x-slow': 3, 'x-fast': 4})

In [19]:
for index, row in lenghtData.iterrows():
  dataframe = pd.DataFrame({'filename' : [], 'script' : []})
  listScripts = {'filename': row['filename'], 'script': ""}
  script_location = scriptDirectory+str(row['filename'])+".txt"
  with open(script_location, "r") as f:
    script = f.read()
  listScripts['filename'] = row['filename']
  listScripts['script'] = script
  dataframe = pd.concat([dataframe, pd.Series(listScripts).to_frame().T], ignore_index=True)
  dataframe.to_csv(writeDirectory+"scripts.csv", mode='a', header=not os.path.exists(writeDirectory+"scripts.csv"))


In [29]:
import re
import os

directory = r"C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\dev\\"  
    
for file in os.listdir(directory+"\stm\\"):
  filename = os.fsdecode(directory+"\stm\\"+file)
  with open(filename, "r") as f:
    long_script = []
    for line in f:
      no_tags = re.sub(r'^.*?>', '', line)
      no_tags = no_tags.replace(" <unk>",',')
      no_tags = no_tags.replace(" '","'")
      long_script.append(no_tags)
    joined_script = "".join(long_script)
    script = joined_script.replace("\n,",".")
    script = script.replace("\n","")
    clean_filename = file.replace(".stm", "")
    text_file = open(directory+"\\txt\\"+ clean_filename +".txt", "w+")
    text_file.write(script)
    text_file.close()
    f.close()

In [30]:
val = [150, 234, 254, 263, 281, 451, 458, 593, 730, 869, 875, 901, 908, 1022, 1168, 1132, 1079, 1242, 1245, 1251]
test = [7, 12, 76, 147, 152, 324, 478, 516, 591, 665, 759, 793, 823, 840, 842, 848, 938, 947, 1063, 1104]
lenghtData = pd.read_csv('C:\\Users\\ptut0\\Documents\\speech_timing\\basline\\data\\clean_baseline.csv')

In [41]:
for index, row in lenghtData.iterrows():
  dataframe_train = pd.DataFrame({'filename' : [],  'word_lengths' : [], 'pause_lengths' : [], 'discrete_lengths' : [],	'discrete_pauses' : [],	'script' : []})
  dataframe_val = pd.DataFrame({'filename' : [],  'word_lengths' : [], 'pause_lengths' : [], 'discrete_lengths' : [],	'discrete_pauses' : [],	'script' : []})
  dataframe_test = pd.DataFrame({'filename' : [],  'word_lengths' : [], 'pause_lengths' : [], 'discrete_lengths' : [],	'discrete_pauses' : [],	'script' : []})
  if row['filename'] in val:
    dataframe_val = pd.concat([dataframe_val, row.to_frame().T], ignore_index=True)
    dataframe_val.to_csv("baseline_val.csv", mode='a', header=not os.path.exists("baseline_val.csv"))
  elif row['filename'] in test:
    dataframe_test = pd.concat([dataframe_test, row.to_frame().T], ignore_index=True)
    dataframe_test.to_csv("baseline_test.csv", mode='a', header=not os.path.exists("baseline_test.csv"))
  else:
    dataframe_train = pd.concat([dataframe_train, row.to_frame().T], ignore_index=True)
    dataframe_train.to_csv("baseline_train.csv", mode='a', header=not os.path.exists("baseline_train.csv"))

In [37]:
print(len(['slow', 'slow', 'fast', 'slow', 'slow', 'slow', 'x-fast', 'x-fast', 'medium', 'fast', 'medium', 'medium', 'medium', 'x-slow', 'slow', 'x-slow', 'fast', 'medium', 'fast', 'x-fast', 'slow', 'slow', 'fast', 'fast', 'slow', 'x-fast', 'x-fast', 'x-fast', 'medium', 'x-fast', 'x-fast', 'medium', 'fast', 'x-slow', 'fast', 'medium', 'fast', 'fast', 'slow', 'slow', 'x-slow', 'x-fast', 'medium', 'slow', 'medium', 'medium', 'medium', 'slow', 'slow', 'x-fast', 'slow', 'slow', 'medium', 'x-fast', 'slow', 'slow', 'fast', 'slow', 'slow', 'x-slow', 'fast', 'x-fast', 'x-slow', 'slow', 'medium', 'slow', 'slow', 'medium', 'medium', 'slow', 'medium', 'x-slow', 'x-fast', 'slow', 'x-fast', 'slow', 'fast', 'x-fast', 'x-slow', 'fast', 'fast', 'fast', 'fast', 'x-slow', 'fast', 'x-fast', 'x-slow', 'x-fast', 'x-slow', 'x-fast', 'x-fast', 'slow', 'medium', 'slow', 'slow', 'slow', 'medium', 'slow', 'fast', 'x-fast', 'fast', 'x-slow', 'x-slow', 'x-slow', 'x-slow', 'slow', 'fast', 'x-fast', 'slow', 'x-fast', 'x-fast', 'medium', 'x-fast', 'x-slow', 'fast', 'fast', 'x-slow', 'fast', 'x-slow', 'slow', 'x-fast', 'slow', 'slow', 'slow', 'fast', 'medium', 'slow', 'fast', 'fast', 'slow', 'medium', 'fast', 'medium', 'x-fast', 'x-slow', 'x-fast', 'slow', 'slow', 'fast', 'fast', 'slow', 'slow', 'medium', 'medium', 'x-slow', 'x-slow', 'slow', 'x-fast', 'slow', 'slow', 'x-fast', 'x-fast', 'medium', 'medium', 'medium', 'x-fast', 'fast', 'x-fast', 'fast', 'x-fast', 'fast', 'medium', 'fast', 'medium', 'fast', 'medium', 'x-slow', 'x-fast', 'x-slow', 'x-fast', 'x-slow', 'x-fast', 'x-fast', 'fast', 'x-slow', 'fast', 'x-slow', 'fast', 'x-fast', 'fast', 'x-fast', 'x-fast', 'x-fast', 'fast', 'medium', 'medium', 'fast', 'fast', 'fast', 'fast', 'fast', 'x-slow', 'x-slow', 'fast', 'medium', 'fast', 'fast', 'x-fast', 'fast', 'x-fast', 'x-slow', 'fast', 'x-slow', 'x-slow', 'x-fast', 'x-slow', 'x-slow', 'fast', 'x-slow', 'slow', 'x-slow', 'slow', 'x-slow', 'x-slow', 'medium', 'medium', 'x-slow', 'fast', 'x-fast', 'x-slow', 'x-slow', 'x-fast', 'x-slow', 'slow', 'x-slow', 'x-slow', 'x-slow', 'slow', 'fast', 'slow', 'x-slow', 'x-fast', 'x-slow', 'slow', 'x-slow', 'x-fast', 'slow', 'slow', 'slow', 'slow', 'medium', 'fast', 'slow', 'x-slow', 'slow', 'x-fast', 'x-slow', 'slow', 'slow', 'slow', 'x-fast', 'x-slow', 'fast', 'x-slow', 'x-slow', 'x-slow', 'x-slow', 'x-slow', 'slow', 'x-slow', 'slow', 'medium', 'x-slow', 'x-slow', 'x-slow', 'slow', 'medium', 'slow', 'slow', 'x-fast', 'fast', 'x-slow', 'fast', 'x-fast', 'x-fast', 'medium', 'medium', 'x-fast', 'slow', 'x-slow', 'fast', 'slow', 'fast', 'x-fast', 'x-fast', 'slow', 'x-slow', 'x-fast', 'x-slow', 'slow', 'fast', 'fast', 'fast', 'fast', 'medium', 'x-fast', 'x-fast', 'medium', 'medium', 'x-slow', 'medium', 'fast', 'x-slow', 'x-fast', 'fast', 'medium', 'fast', 'fast', 'fast', 'slow', 'fast', 'slow', 'x-slow', 'medium', 'medium', 'medium', 'medium', 'medium', 'fast', 'x-fast', 'x-fast', 'fast', 'fast', 'x-fast', 'fast', 'fast', 'x-fast', 'x-slow', 'x-slow', 'slow', 'x-slow', 'medium', 'slow', 'medium', 'fast', 'fast', 'fast', 'x-fast', 'x-slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'x-slow', 'fast', 'medium', 'x-slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'slow', 'x-slow', 'slow', 'slow']))
print(len(['slow', 'medium', 'medium', 'slow', 'fast', 'slow', 'x-fast', 'x-fast', 'slow', 'fast', 'slow', 'fast', 'fast', 'fast', 'x-slow', 'medium', 'medium', 'fast', 'slow', 'x-fast', 'medium', 'medium', 'x-slow', 'x-fast', 'fast', 'x-fast', 'medium', 'x-fast', 'x-fast', 'medium', 'x-fast', 'x-fast', 'fast', 'x-fast', 'medium', 'fast', 'slow', 'x-slow', 'fast', 'slow', 'x-slow', 'x-slow', 'x-slow', 'x-fast', 'x-slow', 'fast', 'medium', 'fast', 'fast', 'slow', 'x-slow', 'slow', 'x-fast', 'x-slow', 'fast', 'x-fast', 'x-fast', 'medium', 'x-fast', 'slow', 'x-fast', 'medium', 'x-slow', 'fast', 'x-fast', 'medium', 'x-slow', 'fast', 'x-fast', 'slow', 'x-fast', 'x-fast', 'medium', 'medium', 'medium', 'x-fast', 'x-slow', 'fast', 'medium', 'slow', 'x-fast', 'medium', 'x-slow', 'fast', 'x-fast', 'fast', 'medium', 'fast', 'fast', 'x-slow', 'medium', 'slow', 'x-slow', 'x-fast', 'slow', 'x-slow', 'x-fast', 'slow', 'medium', 'slow', 'fast', 'fast', 'x-fast', 'medium', 'x-slow', 'fast', 'slow', 'slow', 'fast', 'slow', 'x-fast', 'slow', 'fast', 'x-fast', 'slow', 'slow', 'slow', 'x-slow', 'fast', 'fast', 'x-fast', 'slow', 'medium', 'x-fast', 'x-fast', 'x-slow', 'x-slow', 'medium', 'medium', 'x-slow', 'x-slow', 'x-slow', 'fast', 'x-slow', 'slow', 'x-slow', 'slow', 'fast', 'slow', 'x-fast', 'x-slow', 'medium', 'slow', 'x-fast', 'slow', 'x-slow', 'slow', 'medium', 'medium', 'medium', 'fast', 'x-slow', 'medium', 'x-fast', 'medium', 'medium', 'slow', 'fast', 'x-fast', 'fast', 'slow', 'medium', 'x-fast', 'x-fast', 'slow', 'fast', 'medium', 'slow', 'medium', 'slow', 'slow', 'x-slow', 'x-fast', 'x-fast', 'x-slow', 'medium', 'x-fast', 'fast', 'slow', 'fast', 'slow', 'x-slow', 'slow', 'x-fast', 'fast', 'x-fast', 'slow', 'x-slow', 'x-fast', 'medium', 'medium', 'medium', 'x-slow', 'fast', 'x-slow', 'fast', 'x-slow', 'x-slow', 'fast', 'slow', 'fast', 'x-fast', 'medium', 'x-fast', 'fast', 'medium', 'x-fast', 'x-fast', 'x-slow', 'medium', 'x-slow', 'medium', 'medium', 'x-fast', 'x-fast', 'medium', 'x-slow', 'x-slow', 'medium', 'slow', 'x-slow', 'x-fast', 'fast', 'x-slow', 'x-fast', 'medium', 'slow', 'x-slow', 'x-fast', 'x-fast', 'x-slow', 'fast', 'x-fast', 'slow', 'x-slow', 'medium', 'fast', 'slow', 'fast', 'medium', 'x-fast', 'medium', 'x-fast', 'fast', 'medium', 'fast', 'slow', 'fast', 'x-slow', 'fast', 'x-fast', 'x-slow', 'slow', 'x-fast', 'x-slow', 'medium', 'x-slow', 'medium', 'x-fast', 'slow', 'slow', 'slow', 'fast', 'x-slow', 'slow', 'slow', 'fast', 'slow', 'x-fast', 'medium', 'x-slow', 'fast', 'x-slow', 'medium', 'x-slow', 'x-fast', 'x-slow', 'slow', 'medium', 'medium', 'fast', 'medium', 'fast', 'fast', 'x-fast', 'x-slow', 'x-slow', 'fast', 'x-slow', 'x-fast', 'x-slow', 'x-slow', 'slow', 'slow', 'slow', 'fast', 'medium', 'x-slow', 'medium', 'fast', 'fast', 'slow', 'x-slow', 'slow', 'medium', 'medium', 'slow', 'x-fast', 'x-fast', 'x-fast', 'medium', 'fast', 'x-slow', 'medium', 'x-fast', 'x-slow', 'medium', 'fast', 'medium', 'x-fast', 'slow', 'slow', 'x-slow', 'medium', 'slow', 'x-slow', 'medium', 'x-slow', 'medium', 'medium', 'x-slow', 'slow', 'x-fast', 'fast', 'slow', 'x-fast', 'slow', 'fast', 'medium', 'x-fast', 'slow', 'x-fast', 'fast', 'x-fast', 'fast', 'slow', 'x-slow', 'medium', 'x-fast', 'x-slow', 'x-fast', 'x-fast', 'slow', 'slow', 'fast', 'slow', 'fast', 'fast', 'x-slow', 'slow', 'x-slow', 'medium', 'medium', 'medium', 'x-slow']))
print(len("language spoken by hungarians for which given the fact that cognitive diversity is at least as threatened as biodiversity on this planet few would have imagined much of a future even a century or two ago but there it is j napot, pack i said somebody here must surely know because despite the fact that there aren't that many hungarians to begin with and the further fact that so far as i know there's not a drop of hungarian blood in my veins at every critical juncture of my life. there has been a hungarian friend or mentor there beside me i even have dreams that take place in landscapes i recognize as the landscapes of hungarian films especially the early movies of miklos jancso so how do i explain this mysterious affinity, maybe it's because my native state of south carolina which is not much smaller than present day hungary,. once imagined a future for itself as an independent country and as a consequence of that presumption my hometown was burned to the ground by an invading army an experience. that has befallen many a hungarian town and village throughout its long and troubled history or maybe it's because when i was a teenager back in the' 50s my uncle henry having. denounced the ku klux klan and been bombed for his trouble and had crosses burned in his yard living under death threat took his wife and children to massachusetts for safety and went back to south carolina to face down the klan alone that was a very hungarian thing to do as anyone will attest who remembers, and of course from time to time hungarians have invented their own equivalent of the klan well, it seems to me that this hungarian presence in my life is difficult to account for but ultimately i ascribe it to an admiration for people with a complex moral awareness with a heritage of guilt and defeat matched by defiance and bravado, it's not a typical mindset for most americans, but it is perforce typical of virtually all hungarians so j napot pack. i went back to south carolina after some fifteen years amid the alien corn".split()))
print(len(['x-fast', 'x-fast', 'x-slow', 'x-fast', 'x-fast', 'medium', 'x-slow', 'fast', 'medium', 'x-slow', 'medium', 'medium', 'fast', 'medium', 'medium', 'slow', 'x-slow', 'slow', 'medium', 'x-slow', 'slow', 'medium', 'x-fast', 'medium', 'medium', 'slow', 'medium', 'x-fast', 'x-fast', 'medium', 'fast', 'medium', 'medium', 'slow', 'slow', 'x-slow', 'fast', 'x-slow', 'x-fast', 'x-slow', 'x-slow', 'medium', 'x-slow', 'x-fast', 'medium', 'x-fast', 'fast', 'x-slow', 'slow', 'slow', 'medium', 'slow', 'x-fast', 'x-fast', 'x-slow', 'fast', 'medium', 'x-slow', 'slow', 'fast', 'slow', 'x-slow', 'x-slow', 'x-slow', 'fast', 'x-fast', 'fast', 'slow', 'x-slow', 'slow', 'slow', 'slow', 'medium', 'fast', 'medium', 'x-slow', 'x-slow', 'x-fast', 'medium', 'x-slow', 'fast', 'medium', 'fast', 'medium', 'medium', 'fast', 'slow', 'medium', 'medium', 'x-fast', 'x-fast', 'medium', 'slow', 'medium', 'x-slow', 'x-slow', 'x-fast', 'slow', 'x-slow', 'slow', 'medium', 'x-fast', 'medium', 'slow', 'fast', 'x-slow', 'slow', 'fast', 'x-slow', 'x-fast', 'slow', 'medium', 'x-slow', 'fast', 'x-fast', 'x-slow', 'fast', 'x-slow', 'x-fast', 'x-slow', 'slow', 'medium', 'x-slow', 'x-fast', 'slow', 'fast', 'x-slow', 'medium', 'fast', 'x-slow', 'x-slow', 'fast', 'x-slow', 'x-fast', 'fast', 'x-slow', 'medium', 'fast', 'x-slow', 'x-slow', 'x-slow', 'slow', 'fast', 'slow', 'medium', 'x-fast', 'slow', 'fast', 'x-fast', 'medium', 'fast', 'medium', 'x-slow', 'medium', 'x-slow', 'fast', 'x-fast', 'fast', 'x-fast', 'fast', 'fast', 'x-fast', 'x-fast', 'x-fast', 'slow', 'medium', 'x-fast', 'x-fast', 'fast', 'x-slow', 'slow', 'x-fast', 'medium', 'x-slow', 'x-fast', 'fast', 'fast', 'x-slow', 'x-slow', 'slow', 'x-slow', 'medium', 'medium', 'medium', 'medium', 'slow', 'x-slow', 'x-slow', 'x-fast', 'x-fast', 'slow', 'medium', 'x-fast', 'slow', 'x-slow', 'medium', 'medium', 'x-fast', 'slow', 'medium', 'x-fast', 'x-fast', 'medium', 'medium', 'slow', 'fast', 'slow', 'fast', 'x-slow', 'fast', 'medium', 'x-slow', 'x-fast', 'x-slow', 'x-slow', 'fast', 'fast', 'x-slow', 'x-slow', 'medium', 'x-slow', 'fast', 'slow', 'fast', 'fast', 'fast', 'x-fast', 'slow', 'fast', 'slow', 'x-slow', 'x-slow', 'x-fast', 'x-fast', 'x-slow', 'x-slow', 'fast', 'x-slow', 'fast', 'x-slow', 'x-slow', 'fast', 'x-fast', 'slow', 'fast', 'x-fast', 'slow', 'medium', 'medium', 'fast', 'x-slow', 'x-fast', 'fast', 'medium', 'x-fast', 'x-fast', 'x-fast', 'fast', 'slow', 'slow', 'medium', 'medium', 'fast', 'medium', 'slow', 'slow', 'medium', 'x-fast', 'medium', 'medium', 'medium', 'x-fast', 'x-fast', 'x-fast', 'medium', 'fast', 'fast', 'slow', 'fast', 'x-slow', 'x-fast', 'fast', 'fast', 'x-fast', 'medium', 'fast', 'x-slow', 'slow', 'medium', 'medium', 'x-fast', 'medium', 'medium', 'x-slow', 'fast', 'x-fast', 'slow', 'slow', 'x-fast', 'slow', 'slow', 'fast', 'x-slow', 'medium', 'x-slow', 'fast', 'medium', 'x-slow', 'medium', 'x-fast', 'x-fast', 'fast', 'x-slow', 'x-fast', 'slow', 'x-slow', 'fast', 'x-fast', 'fast', 'medium', 'medium', 'fast', 'slow', 'medium', 'fast', 'slow', 'medium', 'slow', 'medium', 'x-fast', 'x-fast', 'x-fast', 'medium', 'fast', 'x-slow', 'fast', 'slow', 'medium', 'fast', 'medium', 'fast', 'slow', 'x-fast', 'slow', 'slow', 'x-slow', 'fast', 'fast', 'x-slow', 'fast', 'x-slow', 'x-slow', 'fast', 'slow', 'x-slow', 'fast', 'fast', 'x-slow', 'x-fast', 'medium', 'x-fast', 'fast', 'x-fast', 'slow', 'slow']))
print(len(['x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast']))
print(len(['slow', 'slow', 'x-fast', 'slow', 'medium', 'x-slow', 'fast', 'x-fast', 'x-slow', 'x-slow', 'fast', 'fast', 'slow', 'x-fast', 'medium', 'x-fast', 'slow', 'x-fast', 'x-fast', 'x-fast', 'medium', 'slow', 'x-slow', 'fast', 'x-fast', 'slow', 'slow', 'fast', 'x-fast', 'x-slow', 'fast', 'x-fast', 'fast', 'x-fast', 'fast', 'medium', 'fast', 'medium', 'fast', 'x-slow', 'medium', 'slow', 'x-slow', 'x-fast', 'medium', 'x-fast', 'medium', 'x-slow', 'fast', 'x-slow', 'slow', 'slow', 'x-fast', 'x-slow', 'medium', 'fast', 'x-fast', 'fast', 'fast', 'medium', 'x-fast', 'medium', 'slow', 'fast', 'x-fast', 'fast', 'x-slow', 'medium', 'slow', 'x-slow', 'fast', 'x-fast', 'slow', 'medium', 'medium', 'x-fast', 'slow', 'x-fast', 'x-fast', 'x-slow', 'x-fast', 'fast', 'x-slow', 'fast', 'x-fast', 'fast', 'medium', 'x-fast', 'x-fast', 'x-slow', 'medium', 'medium', 'medium', 'x-fast', 'medium', 'x-slow', 'x-fast', 'medium', 'x-slow', 'medium', 'slow', 'fast', 'x-fast', 'medium', 'x-slow', 'fast', 'medium', 'x-slow', 'fast', 'slow', 'x-fast', 'slow', 'x-fast', 'x-fast', 'slow', 'fast', 'medium', 'x-slow', 'x-fast', 'fast', 'fast', 'slow', 'x-fast', 'slow', 'x-slow', 'x-slow', 'medium', 'x-fast', 'x-fast', 'slow', 'medium', 'medium', 'fast', 'x-slow', 'medium', 'medium', 'fast', 'medium', 'x-slow', 'x-fast', 'slow', 'x-fast', 'x-fast', 'x-fast', 'slow', 'slow', 'medium', 'fast', 'medium', 'x-slow', 'medium', 'slow', 'medium', 'x-fast', 'medium', 'x-fast', 'slow', 'x-fast', 'x-fast', 'fast', 'x-slow', 'medium', 'fast', 'x-fast', 'medium', 'x-fast', 'x-fast', 'slow', 'medium', 'slow', 'medium', 'slow', 'x-fast', 'x-fast', 'x-slow', 'fast', 'fast', 'medium', 'slow', 'x-fast', 'slow', 'medium', 'x-slow', 'medium', 'medium', 'x-fast', 'x-fast', 'x-slow', 'medium', 'slow', 'slow', 'fast', 'x-slow', 'fast', 'x-slow', 'medium', 'fast', 'x-slow', 'fast', 'x-fast', 'fast', 'x-fast', 'fast', 'x-fast', 'medium', 'slow', 'x-fast', 'x-fast', 'x-slow', 'medium', 'x-slow', 'x-slow', 'medium', 'slow', 'x-fast', 'medium', 'slow', 'slow', 'x-fast', 'medium', 'x-slow', 'fast', 'fast', 'x-slow', 'fast', 'x-fast', 'x-slow', 'x-slow', 'x-fast', 'fast', 'x-slow', 'medium', 'x-fast', 'x-slow', 'slow', 'medium', 'medium', 'x-slow', 'x-fast', 'slow', 'x-fast', 'slow', 'medium', 'fast', 'fast', 'medium', 'medium', 'x-fast', 'x-slow', 'fast', 'fast', 'x-slow', 'x-slow', 'x-fast', 'x-slow', 'medium', 'fast', 'fast', 'x-fast', 'fast', 'medium', 'slow', 'x-fast', 'x-slow', 'slow', 'slow', 'slow', 'slow', 'medium', 'medium', 'medium', 'x-fast', 'slow', 'x-slow', 'x-slow', 'x-slow', 'medium', 'medium', 'medium', 'slow', 'medium', 'medium', 'fast', 'x-fast', 'x-fast', 'x-slow', 'x-slow', 'fast', 'x-slow', 'fast', 'slow', 'x-slow', 'medium', 'medium', 'fast', 'medium', 'x-slow', 'slow', 'medium', 'x-fast', 'x-fast', 'x-slow', 'x-slow', 'fast', 'x-fast', 'slow', 'slow', 'slow', 'fast', 'fast', 'medium', 'x-slow', 'x-slow', 'x-fast', 'x-fast', 'x-slow', 'fast', 'fast', 'x-fast', 'slow', 'slow', 'x-slow', 'medium', 'x-slow', 'x-slow', 'x-slow', 'x-slow', 'x-slow', 'fast', 'fast', 'slow', 'medium', 'x-fast', 'x-fast', 'x-slow', 'fast', 'slow', 'slow', 'slow', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'slow', 'slow', 'slow', 'slow', 'slow', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'x-fast', 'medium', 'medium', 'medium', 'medium', 'slow', 'x-fast', 'slow', 'slow', 'slow']))

365
365
365
365
365
365


In [21]:
import pandas as pd
import ast
import torch
import string
import numpy as np
from transformers import BertForTokenClassification
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from transformers import BertTokenizerFast
#disable parellelism warning
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

data = pd.read_csv("C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\train\\discrete_word_length.csv")
test_data = pd.read_csv("C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\test\\chunk_length.csv")
dev_data = pd.read_csv("C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\dev\\discrete_word_length.csv")

#ssmlTags = set({'x-slow', 'slow', 'medium', 'fast', 'x-fast', 'x-weak', 'weak', 'medium-p', 'strong', 'x-strong'})
ssmlTags = set({'x-slow', 'slow', 'medium', 'fast', 'x-fast'})

# Map each label into its id representation and vice versa
#labels_to_ids = {'x-slow': 0, 'slow' : 1, 'medium' : 2, 'fast' : 3, 'x-fast': 4,  'x-weak' : 5, 'weak' : 6, 'medium-p' : 7, 'strong' : 8, 'x-strong' : 9}
#ids_to_labels = {0: 'x-slow', 1: 'slow', 2 : 'medium', 3 : 'fast', 4: 'x-fast', 5 : 'x-weak', 6 : 'weak', 7 : 'medium-p', 8 : 'strong', 9 : 'x-strong'}
labels_to_ids = {'x-slow': 0, 'slow' : 1, 'medium' : 2, 'fast' : 3, 'x-fast': 4}
ids_to_labels = {0: 'x-slow', 1: 'slow', 2 : 'medium', 3 : 'fast', 4: 'x-fast'}

def align_label(scripts, labels):
  # test removing all punctuation
  scripts = scripts.translate(str.maketrans('', '', string.punctuation))
  tokenized_inputs = tokenizer(scripts, padding='max_length', max_length=512, truncation=True)

  pausePunct = ',!?.'
  pauseTracker = 0

  word_ids = tokenized_inputs.word_ids()
  input_ids = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
  #print(f"labels: {len(labels)}")
  #print(f"pause labels: {len(pause_labels)}")
  #print(f"script: {len(scripts.split())}")
  #print(f" punct : {scripts.count('.') + scripts.count('!') + scripts.count('?') + scripts.count(',')}")

  previous_word_idx = None
  label_ids = []

  # to move forward at punctuation
  m = 0

  for i, word_idx in enumerate(word_ids):
    if word_idx is None:
      label_ids.append(-100)
    # at punctuation
    elif input_ids[i] in string.punctuation:
      label_ids.append(-100)
      m += 1
      # for a pause add a pause tag
      #if input_ids[i] in pausePunct:
      #  if word_ids[i + 1] != None:
      #    label_ids.append(labels_to_ids[pause_labels[pauseTracker]])
      #    pauseTracker += 1
      #    m += 1
      #  # end of sentence has no pause
      #  else:
      #    label_ids.append(-100)
      #    m += 1
      ## other punctuation no tag
      #else:
      #  label_ids.append(-100)
      #  m += 1
    # combine words with apostrophe to one word
    elif input_ids[i-1] == "'":
      label_ids.append(-100)
      m +=1
    # new word
    elif word_idx != previous_word_idx:
      try:
        label_ids.append(labels_to_ids[labels[word_idx-m]])
      except:
        label_ids.append(-100)
    # word with multiple tags
    else:
      print(f"index : {word_idx}")
      print(f" word ids: {word_ids}")
      print(f"label len : {len(labels)}")
      print(f" script: {input_ids}")
      label_ids.append(labels_to_ids[labels[word_idx-m]])
    previous_word_idx = word_idx
  return label_ids

In [15]:
script = "i'd begin with a scene of war, there was little to warn of the danger ahead the iraqi insurgent had placed the ied an improvised explosive device along the side of the road with great care by two. thousand and six there were more than two thousand five hundred of these attacks every single month, and they were the leading cause of casualties among american soldiers and iraqi civilians the team that was hunting for this ied is called an eod team explosives ordinance disposaland they're the pointy end of the spear in the american effort to suppress these roadside bombs each eod team goes out on about six. hundred of these bomb calls every year defusing about two bombs a day perhaps the best sign of how valuable they are to the war effort is that the iraqi insurgents put a $ fifty thousand bounty on the head of a single eod soldier unfortunately, this particular call would not end well, by the time the soldier advanced close enough to see the telltale wires of the bomb, it exploded in a wave of flame,. close you are and how much explosive has been packed into that bomb, it can cause death or injury you have to be as far as fifty yards away to escape that the blast is so strong it can even break your limbs even if you're not hit that soldier had been on top of the bomb, and so when the rest of the team advanced they found little left, and that night the unit's commander did a sad duty, and he wrote a condolence letter back to the united states and he talked about how hard the loss had been on his unit about the fact that they had lost their. soldier a soldier who had saved their lives many a time, and he apologized for not being able to bring them home, but then he talked up the silver lining that he took away from the loss at least as he wrote when a robot dies, you don't have to write a letter to its mother that scene sounds like science fiction but is battlefield reality already."
labels = ['x-fast', 'medium', 'medium', 'x-fast', 'slow', 'fast', 'x-slow', 'fast', 'fast', 'slow', 'x-fast', 'x-slow', 'fast', 'x-fast', 'medium', 'medium', 'x-fast', 'fast', 'x-fast', 'medium', 'x-slow', 'medium', 'x-slow', 'fast', 'medium', 'medium', 'slow', 'medium', 'x-fast', 'slow', 'fast', 'x-fast', 'slow', 'fast', 'slow', 'x-slow', 'fast', 'medium', 'fast', 'fast', 'x-slow', 'medium', 'x-fast', 'x-slow', 'x-slow', 'slow', 'slow', 'x-slow', 'x-slow', 'medium', 'medium', 'slow', 'fast', 'slow', 'x-slow', 'medium', 'x-fast', 'fast', 'x-fast', 'medium', 'slow', 'fast', 'medium', 'medium', 'fast', 'slow', 'slow', 'fast', 'slow', 'fast', 'slow', 'fast', 'fast', 'medium', 'fast', 'slow', 'x-slow', 'x-fast', 'medium', 'fast', 'x-slow', 'x-slow', 'medium', 'medium', 'medium', 'slow', 'x-fast', 'medium', 'fast', 'x-fast', 'x-fast', 'x-slow', 'fast', 'slow', 'fast', 'medium', 'fast', 'medium', 'medium', 'slow', 'x-slow', 'slow', 'x-slow', 'x-slow', 'slow', 'medium', 'fast', 'medium', 'x-slow', 'slow', 'x-fast', 'slow', 'x-slow', 'x-slow', 'fast', 'x-slow', 'x-fast', 'medium', 'medium', 'x-slow', 'x-fast', 'x-slow', 'medium', 'fast', 'slow', 'x-slow', 'medium', 'slow', 'slow', 'fast', 'medium', 'fast', 'x-fast', 'medium', 'medium', 'fast', 'slow', 'x-fast', 'fast', 'slow', 'slow', 'fast', 'x-slow', 'slow', 'slow', 'x-fast', 'fast', 'x-slow', 'fast', 'x-fast', 'x-slow', 'x-slow', 'x-slow', 'x-slow', 'x-fast', 'x-slow', 'x-fast', 'fast', 'slow', 'medium', 'x-slow', 'medium', 'fast', 'slow', 'x-fast', 'x-slow', 'medium', 'fast', 'slow', 'x-fast', 'medium', 'fast', 'x-slow', 'slow', 'medium', 'fast', 'x-slow', 'fast', 'x-slow', 'x-fast', 'medium', 'x-slow', 'x-fast', 'x-fast', 'slow', 'medium', 'fast', 'fast', 'fast', 'slow', 'x-slow', 'x-fast', 'medium', 'slow', 'medium', 'fast', 'x-slow', 'medium', 'fast', 'slow', 'x-slow', 'x-fast', 'x-slow', 'x-fast', 'fast', 'x-fast', 'fast', 'fast', 'slow', 'x-fast', 'slow', 'fast', 'slow', 'x-fast', 'x-slow', 'fast', 'fast', 'x-slow', 'x-fast', 'fast', 'x-slow', 'x-fast', 'fast', 'x-slow', 'medium', 'fast', 'x-slow', 'medium', 'x-fast', 'fast', 'slow', 'slow', 'medium', 'x-slow', 'x-fast', 'x-slow', 'medium', 'x-slow', 'medium', 'x-fast', 'slow', 'medium', 'medium', 'fast', 'fast', 'slow', 'fast', 'x-fast', 'slow', 'x-slow', 'x-fast', 'slow', 'slow', 'x-slow', 'fast', 'medium', 'slow', 'x-fast', 'x-slow', 'medium', 'x-fast', 'x-fast', 'x-slow', 'x-slow', 'x-fast', 'x-fast', 'slow', 'x-fast', 'x-slow', 'fast', 'fast', 'x-fast', 'x-fast', 'slow', 'medium', 'x-fast', 'x-fast', 'x-slow', 'slow', 'x-fast', 'slow', 'x-fast', 'slow', 'fast', 'fast', 'fast', 'x-fast', 'slow', 'x-fast', 'x-fast', 'x-slow', 'x-fast', 'medium', 'fast', 'x-slow', 'fast', 'x-slow', 'x-fast', 'x-slow', 'x-fast', 'x-fast', 'slow', 'fast', 'x-slow', 'medium', 'x-fast', 'x-slow', 'fast', 'x-fast', 'x-slow', 'x-fast', 'medium', 'medium', 'slow', 'x-fast', 'medium', 'medium', 'slow', 'fast', 'fast', 'fast', 'slow', 'fast', 'fast', 'slow', 'medium', 'x-fast', 'fast', 'medium', 'slow', 'x-fast', 'x-fast', 'x-slow', 'fast', 'slow', 'fast', 'fast', 'slow', 'medium', 'x-fast', 'x-slow', 'slow', 'fast', 'slow', 'medium', 'x-fast', 'medium', 'x-fast', 'x-slow', 'x-fast', 'medium', 'x-slow', 'fast', 'x-slow', 'x-slow', 'slow', 'x-slow', 'x-slow', 'x-fast', 'fast', 'x-slow', 'slow', 'slow', 'x-fast']

In [19]:
print(len(script.split()))
print(len(labels))

364
364


In [22]:
print(align_label(script,labels))

index : 17
 word ids: [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 17, 18, 18, 18, 19, 20, 21, 22, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 64, 64, 65, 66, 67, 67, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 76, 77, 78, 79, 80, 80, 81, 82, 83, 84, 84, 85, 85, 86, 87, 87, 88, 89, 90, 91, 92, 93, 94, 94, 94, 95, 96, 97, 98, 99, 99, 100, 101, 102, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 138, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196

In [27]:
test = [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 14, 15, 16, 17, 18, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 63, 64, 65, 66, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 111, 112, 113, 114, 115, 116, 117, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 129, 130, 131, 132, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 156, 156, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 166, 167, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 246, 247, 248, 249, 250, 251, 251, 252, 252, 253, 254, 255, 256, 257, 257, 258, 259, 260, 261, 262, 263, 264, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [50]:
print(len([1.11, 0.86, 0.16, 0.88, 0.23, 0.11, 0.47, 0.74, 0.18, 0.46, 0.12, 0.73, 0.3, 0.62, 0.19, 0.71, 0.15, 0.35, 0.33, 0.26, 0.54, 0.24, 0.53, 0.14, 0.68, 0.22, 0.8, 0.16, 0.75, 0.21, 0.7, 0.35, 0.54, 0.69, 0.42, 0.17, 0.43, 0.19, 0.09, 0.12, 0.28, 0.72, 0.2, 0.05, 0.89, 0.77, 0.67, 0.22, 0.28, 0.58, 0.1, 0.18, 0.11, 0.17, 0.21, 0.27, 0.11, 0.61, 0.2, 0.09, 0.37, 0.46, 0.23, 0.16, 0.14, 0.61, 0.19, 0.18, 0.34, 0.2, 0.22, 0.68, 0.26, 0.32, 0.17, 0.13, 0.3, 0.27, 0.51, 0.17, 0.63, 0.56, 0.18, 0.15, 0.62, 0.67, 0.2, 0.89, 0.2, 0.12, 0.25, 0.65, 0.3, 0.26, 0.47, 0.15, 0.18, 0.09, 0.06, 0.23, 0.29, 0.18, 0.14, 0.29, 'NA', 0.17, 0.57, 0.22, 0.12, 0.2, 0.47, 0.36, 0.28, 0.13, 0.51, 0.15, 0.54, 0.2, 0.12, 0.19, 'NA', 0.09, 0.14, 0.17, 0.19, 0.07, 0.39, 0.15, 0.25, 0.46, 0.16, 0.28, 0.28, 0.08, 0.25, 0.58, 0.11, 0.16, 0.56, 0.17, 0.44, 0.59, 0.17, 0.28, 0.25, 0.13, 0.23, 0.47, 0.25, 0.22, 0.33, 0.26, 0.26, 0.59, 0.67, 0.25, 0.12, 0.21, 0.13, 0.1, 0.47, 0.32, 0.08, 0.28, 0.42, 0.25, 0.44, 0.61, 0.38, 0.19, 0.21, 0.46, 0.53, 0.45, 0.61, 0.13, 0.26, 0.14, 0.13, 0.34, 0.29, 0.23, 0.33, 0.57, 0.23, 0.18, 0.35, 0.57, 0.19, 0.52, 0.08, 0.25, 0.15, 0.31, 0.8, 0.18, 0.15, 0.55, 0.24, 0.11, 0.2, 0.28, 0.19, 0.27, 0.23, 0.1, 0.78, 0.14, 0.27, 0.15, 0.19, 0.11, 0.14, 0.69, 0.21, 0.23, 0.22, 0.27, 0.42, 0.13, 0.1, 0.65, 0.15, 0.31, 0.13, 0.19, 0.48, 0.12, 0.22, 0.19, 0.19, 0.17, 0.46, 0.2, 0.14, 0.07, 0.17, 0.31, 0.08, 0.19, 0.13, 0.12, 0.09, 0.37, 0.54, 0.08, 0.1, 0.38, 0.12, 0.12, 0.17, 0.21, 0.19, 0.24, 0.5, 0.14, 0.34, 0.15, 0.5, 0.2, 0.16, 0.05, 0.14, 0.11, 0.79, 0.14, 0.37, 0.14, 0.16, 0.52, 0.23, 0.15, 0.07, 0.48, 0.22, 0.23, 0.07, 0.61, 0.15, 0.13, 0.17, 0.19, 'NA', 0.2, 0.2, 0.17, 0.72, 0.18, 0.27, 0.07, 0.18, 0.08, 0.15, 0.07, 0.53, 0.15, 0.16, 0.48, 0.13, 0.25, 0.15, 0.15, 0.21, 0.15, 0.7, 0.13, 0.47, 0.21, 0.55, 0.25, 0.19, 0.06, 0.46, 0.13, 0.17, 0.34, 0.32, 0.18, 0.05, 0.24, 0.08, 0.08, 0.29, 0.32, 0.34, 0.31, 0.36, 0.29, 0.55, 0.15, 0.17, 0.4, 0.16, 0.19, 0.54, 0.13, 0.09, 0.09, 0.17, 0.29, 0.56, 0.12, 0.18, 0.05, 0.3, 0.28, 0.52, 0.65, 0.14, 0.04, 0.14, 0.26, 0.33, 0.48, 0.14, 0.15, 0.16, 0.11, 0.48, 0.54, 0.12, 0.12, 0.45, 0.22, 0.22, 0.37, 0.14, 0.26]))

368


In [65]:
data = pd.read_csv("C:\\Users\\ptut0\\Documents\\speech_timing\\TEDLIUM_release-3\\speaker-adaptation\\test\\chunk_length.csv")

for index, row in data.iterrows():
    print(len(ast.literal_eval(row['discrete_lengths'])) == len(row['script'].split()))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
